# Web Scrapers

In [1]:
import requests
import urllib.request
import time
import re
from bs4 import BeautifulSoup
import pandas as pd
import string

## Fantasy Pros

In [2]:
fantasy_pros_urls = ['https://www.fantasypros.com/nfl/projections/qb.php?week=1',
                    'https://www.fantasypros.com/nfl/projections/rb.php?week=1',
                    'https://www.fantasypros.com/nfl/projections/wr.php?week=1',
                    'https://www.fantasypros.com/nfl/projections/te.php?week=1',
                    'https://www.fantasypros.com/nfl/projections/k.php?week=1',
                    'https://www.fantasypros.com/nfl/projections/dst.php?week=1']

In [3]:
fantasy_pros_players = []
fantasy_pros_points = []
fantasy_pros_positions = []

for url in fantasy_pros_urls:
    position = (url.split('https://www.fantasypros.com/nfl/projections/'))[1].split('.php?week=1')[0]
    print("Running Code for {}".format(url))
    response = requests.get(url)
    print(response)
    soup = BeautifulSoup(response.text, "html.parser")
    
    tbl = soup.find('table')
    rows = tbl.findAll('tr')
    for row in rows:
        c = 0
        for names in row.find_all(class_='player-name'):
            fantasy_pros_players.append(names.text)
        for points in row.find_all('td', attrs = {'class' : 'center', 'data-sort-value': True }):
            fantasy_pros_positions.append(position)
            fantasy_pros_points.append(points.text)

    print("Total Number of Players: {}".format(len(fantasy_pros_players)))
    print("Total Numbef of Scores: {}".format(len(fantasy_pros_points)))
    

Running Code for https://www.fantasypros.com/nfl/projections/qb.php?week=1
<Response [200]>
Total Number of Players: 82
Total Numbef of Scores: 82
Running Code for https://www.fantasypros.com/nfl/projections/rb.php?week=1
<Response [200]>
Total Number of Players: 238
Total Numbef of Scores: 238
Running Code for https://www.fantasypros.com/nfl/projections/wr.php?week=1
<Response [200]>
Total Number of Players: 452
Total Numbef of Scores: 452
Running Code for https://www.fantasypros.com/nfl/projections/te.php?week=1
<Response [200]>
Total Number of Players: 575
Total Numbef of Scores: 575
Running Code for https://www.fantasypros.com/nfl/projections/k.php?week=1
<Response [200]>
Total Number of Players: 605
Total Numbef of Scores: 605
Running Code for https://www.fantasypros.com/nfl/projections/dst.php?week=1
<Response [200]>
Total Number of Players: 637
Total Numbef of Scores: 637


## Create Table

In [4]:
d = {'Players':fantasy_pros_players, 'Position':fantasy_pros_positions, 'fantasy_pros':fantasy_pros_points}
df = pd.DataFrame(d)
df.Players = df['Players'].str.replace('[{}]'.format(string.punctuation), '')
df.Players = df['Players'].str.lower()

## My Teams

In [64]:
wide_open_spaces = ['carson wentz', 'deandre hopkins', 'julio jones', 'julian edelman', 'leonard fournette', 
                    'james white', 'eric ebron', 'calvin ridley', 'jordan howard', 'ben roethlisberger', 
                    'chris thompson', 'dion lewis', 'jordan reed', 'dk metcalf', 'robbie gould', 'houston']

In [65]:
brees_up_my_dress = ['baker mayfield', 'deandre hopkins', 'michael thomas', 'joe mixon', 'josh jacobs',
                     'zach ertz', 'ty hilton', 'david montgomery', 'drew brees', 'robby anderson', 'tarik cohen',
                     'derrius guice', 'eric ebron', 'kaimi fairbairn', 'los angeles']

## Find Players

### Functions

In [66]:
def create_team_df(team_name):
    team_df = pd.DataFrame()
    for player in team_name:
        temp = df[df['Players'].str.contains(player)]
        if temp.shape[0] > 0:
            team_df = pd.concat([team_df, temp])
            temp = pd.DataFrame()
        else:
            print("Player {} not found".format(player))
    return team_df

### Get Teams

In [67]:
WOS = create_team_df(wide_open_spaces)

In [68]:
WOS

,Players,Position,fantasy_pros
6,carson wentz,qb,18.1
238,deandre hopkins,wr,11.2
240,julio jones,wr,11.0
254,julian edelman,wr,8.7
93,leonard fournette,rb,12.1
111,james white,rb,8.0
470,eric ebron,te,3.6
264,calvin ridley,wr,7.3
120,jordan howard,rb,6.4
13,ben roethlisberger,qb,17.4


In [70]:
BUMD = create_team_df(brees_up_my_dress)

In [71]:
BUMD

,Players,Position,fantasy_pros
9,baker mayfield,qb,17.8
238,deandre hopkins,wr,11.2
243,michael thomas,wr,10.4
96,joe mixon,rb,11.0
97,josh jacobs,rb,10.9
454,zach ertz,te,7.1
262,ty hilton,wr,7.3
100,david montgomery,rb,10.0
5,drew brees,qb,18.4
270,robby anderson,wr,7.0


### Print Lineup

In [82]:
def get_lineup(df, col):
    idx = df.groupby(['Position'])[col].transform(max) == df[col]
    final = WOS[idx]
    return final

In [83]:
get_lineup(WOS, 'fantasy_pros')

,Players,Position,fantasy_pros
6,carson wentz,qb,18.1
254,julian edelman,wr,8.7
111,james white,rb,8.0
470,eric ebron,te,3.6
471,jordan reed,te,3.6
582,robbie gould,k,8.6
636,houston texans,dst,4.6
